In [1]:
import sys
import time
from pyscf import gto, scf, mcscf, mp, cc
from pyscf import lo
from pyscf.tools import molden, cubegen
from pyscf.geomopt.geometric_solver import optimize
import numpy
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper

## Molecular System (Mol)
We build a molecule object hopping that we manage to make a function to translate the information
The molecule is Ethene

In [2]:
mol = gto.Mole()
mol.atom = [['C',         (-0.66720,        0.00000,        0.00000)],
['C',          (0.66720,        0.00000,        0.00000)],
['H',         (-1.22130,       -0.92900,        0.07080)],
['H',         (-1.22120,        0.92900,       -0.07080)],
['H',          (1.22130,        0.92900,       -0.07080)],
['H',          (1.22130,       -0.92900,        0.07080)]]
mol.basis = 'sto-3g'
mol.symmetry = True
mol.spin = 0
mol.charge = 0
mol.unit = 'A'
mol.verbose = 5
mol.output = '/Users/rcp/Documents/QiskitHackatonEurope/Ethene_Results.txt'
mol.build()

overwrite output file: /Users/rcp/Documents/QiskitHackatonEurope/Ethene_Results.txt


## Classical Calculations of Mol

In [4]:
def HF_std(mol):
    t0=time.clock()
    _=scf.RHF(mol).run()
    with open('mo_HF.molden', 'w') as f1:
        molden.header(mol, f1) 
        molden.orbital_coeff(mol, f1, _.mo_coeff, ene=_.mo_energy, occ=_.mo_occ)
    t1=time.clock()
    cpu_time = t1-t0
    print('CPU Time (s): %s' % (cpu_time))
    print('Dipole moment %s' %(_.dip_moment()))
    print('Energy (a.u.) %5.6f' %(_.e_tot))


In [5]:
def DFT_std(mol):
    t0=time.clock()
    _=scf.RKS(mol).run()
    with open('mo_DFT.molden', 'w') as f1:
        molden.header(mol, f1) 
        molden.orbital_coeff(mol, f1, _.mo_coeff, ene=_.mo_energy, occ=_.mo_occ)
    t1=time.clock()
    cpu_time = t1-t0
    print('CPU Time (s): %s' % (cpu_time))
    print('Dipole moment %s' %(_.dip_moment()))
    print('Energy (a.u.) %5.6f' %(_.e_tot))

In [6]:
def CC_std(mol):
    t0=time.clock()
    _=scf.HF(mol).run()
    _=cc.CCSD(_).run()
    t1=time.clock()
    cpu_time = t1-t0
    print('CPU Time (s): %s' % (cpu_time))
    print('Energy (a.u.) %5.6f' %(_.e_tot))

In [14]:
def CASSCF_std(mol, nelec, nhole):
    t0=time.clock()
    _=scf.HF(mol).run()
    _=mcscf.CASSCF(_,nelec,nhole).run()
    with open('mo_CASSCF.molden', 'w') as f1:
        molden.header(mol, f1) 
        molden.orbital_coeff(mol, f1, _.mo_coeff, ene=_.mo_energy, occ=_.mo_occ)
    t1=time.clock()
    cpu_time = t1-t0
    print('CPU Time (s): %s' % (cpu_time))
    print('Energy (a.u.) %5.6f' %(_.e_tot))

In [15]:
def opt_std_HF(mol):
    _ = scf.RHF(mol).run()
    _ = optimize(_, maxsteps=1000)
    _ = scf.RHF(_).run()
    return _.mol() 

## Fermion to Qubit Mapping of Mol

In [27]:
def F_2_Q(mol):
    t0=time.clock()
    drv= PySCFDriver(molecule=mol)
    es_drv = ElectronicStructureProblem(drv)
    snd_q_op = es_drv.second_q_ops()
    qbt_conv = QubitConverter(mapper=JordanWignerMapper())
    qbt_op = qbt_conv.convert(snd_q_op[0])
    t1=time.clock()
    cpu_time = t1-t0
    print('CPU Time (min): %s' % (cpu_time/60))
    return es_drv, qbt_conv
    #return [snd_q_op, qbt_op]

In [28]:
Ethene = mol
#Ethene_HF =HF_std(Ethene)
#Ethene_DFT =DFT_std(Ethene)
#Ethene_CC =CC_std(Ethene)
#Ethene_CASSCF =CASSCF_std(Ethene,2,2)
#Ethene_F_2_Q = F_2_Q(mol)

PySCF kernel() converged: True, e(hf): -77.07229614334867HF Electronic dipole moment: [-1.7719e-04  9.4000e-07 -7.0000e-08]Nuclear dipole moment: [0.00018897 0.         0.        ]Total dipole moment: [ 1.178e-05  9.400e-07 -7.000e-08]

CPU Time (min): 6.993357183333332


## Native Calculation

In [ ]:
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.providers.aer import StatevectorSimulator
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import VQEUCCFactory
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.algorithms import NumPyMinimumEigensolverFactory
from qiskit_nature.problems.second_quantization import VibrationalStructureProblem
from qiskit_nature.mappers.second_quantization import DirectMapper

In [31]:
def Q_GS(mol):
    t0=time.clock()
    drv= PySCFDriver(molecule=mol)
    es_drv = ElectronicStructureProblem(drv)
    snd_q_op = es_drv.second_q_ops()
    qbt_conv = QubitConverter(mapper=JordanWignerMapper())
    qbt_op = qbt_conv.convert(snd_q_op[0])
    quantum_instance = QuantumInstance(backend = Aer.get_backend('statevector_simulator'))
    vqe_solver = VQEUCCFactory(quantum_instance)
    tl_circuit = TwoLocal(rotation_blocks = ['h', 'rx'], entanglement_blocks = 'cz',
                          entanglement='full', reps=3, parameter_prefix = 'y')
    tl_circuit.draw(output='mpl')
    calc_vqe = GroundStateEigensolver(qbt_conv, vqe_solver)
    res_vqe_solver = calc_vqe.solve(es_drv)
    print(res_vqe_solver)
    calc_numpy = GroundStateEigensolver(qubit_converter, NumPyMinimumEigensolver())
    res_numpy_solver = calc_numpy.solve(es_problem)
    print(res_numpy_solver)
    t1=time.clock()
    cpu_time = t1-t0
    print('CPU Time (min): %s' % (cpu_time/60))
    

PySCF kernel() converged: True, e(hf): -77.07229614334867HF Electronic dipole moment: [-1.7719e-04  9.4000e-07 -7.0000e-08]Nuclear dipole moment: [0.00018897 0.         0.        ]Total dipole moment: [ 1.178e-05  9.400e-07 -7.000e-08]Overwriting the number of shots in the quantum instance with the settings from the backend.
Qiskit Terra version: 0.17.0
Backend: 'statevector_simulator (AerProvider)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cswap', 'csx', 'cu1', 'cu2', 'cu3', 'cx', 'cy', 'cz', 'delay', 'diagonal', 'h', 'id', 'initialize', 'mcp', 'mcr', 'mcrx', 'mcry', 'mcrz', 'mcswap', 'mcsx', 'mcu1', 'mcu2', 'mcu3', 'mcx', 'mcy', 'mcz', 'multiplexer', 'p', 'pauli', 'r', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z', 'kraus', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_density_matrix', 'save_expval', 'save_probs', 'save_probs_ket', 'save_state', 'save_statevector', 'set_sta

KeyboardInterrupt: 

Overwriting the number of shots in the quantum instance with the settings from the backend.
Qiskit Terra version: 0.17.0
Backend: 'statevector_simulator (AerProvider)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cswap', 'csx', 'cu1', 'cu2', 'cu3', 'cx', 'cy', 'cz', 'delay', 'diagonal', 'h', 'id', 'initialize', 'mcp', 'mcr', 'mcrx', 'mcry', 'mcrz', 'mcswap', 'mcsx', 'mcu1', 'mcu2', 'mcu3', 'mcx', 'mcy', 'mcz', 'multiplexer', 'p', 'pauli', 'r', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z', 'kraus', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_density_matrix', 'save_expval', 'save_probs', 'save_probs_ket', 'save_state', 'save_statevector', 'set_statevector', 'kraus', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_density_matrix', 'save_expval', 'save_probs', 'save_probs_ket', 'save_state', 'save_statevector', 'set_statevector', 'kraus', 'roerror', 'save_amplitudes', '

CPU Time (min): 0.0006156166666661041


Overwriting the number of shots in the quantum instance with the settings from the backend.
Qiskit Terra version: 0.17.0
Backend: 'statevector_simulator (AerProvider)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cswap', 'csx', 'cu1', 'cu2', 'cu3', 'cx', 'cy', 'cz', 'delay', 'diagonal', 'h', 'id', 'initialize', 'mcp', 'mcr', 'mcrx', 'mcry', 'mcrz', 'mcswap', 'mcsx', 'mcu1', 'mcu2', 'mcu3', 'mcx', 'mcy', 'mcz', 'multiplexer', 'p', 'pauli', 'r', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z', 'kraus', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_density_matrix', 'save_expval', 'save_probs', 'save_probs_ket', 'save_state', 'save_statevector', 'set_statevector', 'kraus', 'roerror', 'save_amplitudes', 'save_amplitudes_sq', 'save_density_matrix', 'save_expval', 'save_probs', 'save_probs_ket', 'save_state', 'save_statevector', 'set_statevector', 'kraus', 'roerror', 'save_amplitudes', '

CPU Time (min): 0.0010080999999994825


NameError: name 'qbt_conv' is not defined

In [ ]:
from qiskit_nature.drivers import GaussianForcesDriver
from qiskit_nature.algorithms import NumPyMinimumEigensolverFactory
from qiskit_nature.problems.second_quantization import VibrationalStructureProblem
from qiskit_nature.mappers.second_quantization import DirectMapper

drv= PySCFDriver(molecule=mol)

vib_problem = VibrationalStructureProblem(drv, num_modals=2, truncation_order=2)

qubit_converter = QubitConverter(DirectMapper())

solver_without_filter = NumPyMinimumEigensolverFactory(use_default_filter_criterion=False)
solver_with_filter = NumPyMinimumEigensolverFactory(use_default_filter_criterion=True)

gsc_wo = GroundStateEigensolver(qubit_converter, solver_without_filter)
result_wo = gsc_wo.solve(vib_problem)

gsc_w = GroundStateEigensolver(qubit_converter, solver_with_filter)
result_w = gsc_w.solve(vib_problem)

print(result_wo)
print('\n\n')
print(result_w)